In [ ]:
!pip3 install --upgrade pip

In [ ]:
!pip3 install -r requirements.txt

In [ ]:
!pip install opencv-contrib-python-headless==4.8.0.76

In [32]:
import tarfile
import os
from pathlib import Path

source_path = 'ood_enabler.tar.gz'
tar = tarfile.open(source_path, "r:gz")
extract_dir = os.path.join('.', '{}'.format(Path(source_path).with_suffix('')).split('/')[-1].split('.tar')[0])

os.makedirs(extract_dir, exist_ok=True)
tar.extractall(extract_dir)
tar.close()

In [2]:
import torch
import torchvision
import os
import sys
from tempfile import TemporaryDirectory

ood_path = os.path.abspath('.')
if ood_path not in sys.path:
    sys.path.append(ood_path)
    
from ood_enabler.ood_enabler import OODEnabler
from ood_enabler.model_wrapper.pytorch import PytorchWrapper
from ood_enabler.storage.model_store import ModelStore
from ood_enabler.storage.local_storage import FileSystemStorage
from ood_enabler.data.pytorch_image_data_handler import PytorchImageDataHandler
from ood_enabler.util.constants import SavedModelFormat
from ood_enabler.util.archiver import archive

In [3]:
import glob
import json
import os
from pathlib import Path
import requests
import tarfile
from urllib.parse import urlparse
from ast import literal_eval as make_tuple

data_dir_name = 'in_data/in_distribution.zip'
model_path = 'model/model.pt'


In [4]:
local_store = FileSystemStorage()
#ds_metadata = {'img_height': 224, 'img_width': 224, 'batch_size': 32, 'normalize': ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])}
ds_metadata = {'img_height': int(os.getenv("img_height")), 'img_width': int(os.getenv("img_width")), 
               'batch_size': int(os.getenv("batch_size", 32)), 'normalize': os.getenv("normalize")}

if isinstance(ds_metadata['normalize'], str):
    ds_metadata['normalize'] = make_tuple(ds_metadata['normalize'])
    
else:
    ds_metadata['normalize'] = int(ds_metadata['normalize'])
    
data_handler = PytorchImageDataHandler()
data_handler.load_dataset(local_store, data_dir_name, '.', ds_metadata)

In [18]:
model_store = ModelStore.from_filesystem()
model_metadata = {'type': os.getenv("model_type"), 'arch': os.getenv("model_arch"), 'ood_thresh_percentile': 20}

model = model_store.load(model_metadata, model_path)


Number of layers for model from path:52
returning base model  ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True,

In [19]:
OODEnabler.ood_enable(model, data_handler)

adding OOD layer pytorch


Inference progress: 100%|██████████| 13/13 [02:13<00:00, 10.25s/it]

set_threshold for given data handler:Parameter containing:
tensor([0.8423])



/opt/app-root/lib64/python3.9/site-packages/numpy/lib/function_base.py:3700: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  a = np.asanyarray(a)
/opt/app-root/lib64/python3.9/site-packages/numpy/lib/function_base.py:3700: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


In [35]:
model_store.upload(model, 'model/', saved_model_format=SavedModelFormat.ONNX)

'model/'